In [1]:
import urllib.request
import json
import pandas as pd 
import numpy as np
import datetime as dt

In [2]:
train = pd.read_csv('train_after_processing_weatheropt3.csv', index_col=0, encoding = 'cp949') 
test = pd.read_csv('test_after_processing_weatheropt3.csv', index_col=0, encoding = 'cp949')

## 1번(trend) 데이터 값

#### 1years_ago_trend
- 기간: 2018/01/01~2018/12/31
- 기간 중 train과 test set에 있는 모든 item의 정규화된 검색량

In [4]:
items = train['브랜드'].append(test['브랜드']).dropna().unique()
item_df = pd.DataFrame(items, columns = ["브랜드"])
item_df.head()

,브랜드
0,테이트
1,오모떼
2,CERINI by PAT
3,보코
4,크로커다일


In [6]:
import os
import sys
import urllib.request


def datalab_api(keywords):
    client_id = "zkIiC4UfvU1jEWWyp8_2"
    client_secret = "enaM8m_1z7"
    url = "https://openapi.naver.com/v1/datalab/search";
    body = {"startDate":"2018-01-01"
    ,"endDate":"2018-12-31"
    ,"timeUnit":"date"
    ,"keywordGroups":[
        {"groupName":str(keywords[0]),"keywords":[str(keywords[0])]},
        {"groupName":str(keywords[1]),"keywords":[str(keywords[1])]},
        {"groupName":str(keywords[2]),"keywords":[str(keywords[2])]},
        {"groupName":str(keywords[3]),"keywords":[str(keywords[3])]},
        {"groupName":str(keywords[4]),"keywords":[str(keywords[4])]},
    ]
    };
    body = json.dumps(body)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_obj = json.loads(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
        
    df1 = pd.DataFrame(json_obj['results'][0]['data']);
    df1.columns=['기간',str(keywords[0])]; 
    df2 = pd.DataFrame(json_obj['results'][1]['data']);
    df2.columns=['기간',str(keywords[1])]; 
    df3 = pd.DataFrame(json_obj['results'][2]['data']);
    df3.columns=['기간',str(keywords[2])]; 
    df4 = pd.DataFrame(json_obj['results'][3]['data']);
    df4.columns=['기간',str(keywords[3])]; 
    df5 = pd.DataFrame(json_obj['results'][4]['data']);
    df5.columns=['기간',str(keywords[4])]; 
    
    df2= df1.merge(df2,how='outer', on='기간').fillna(0)
    df3= df2.merge(df3,how='outer', on='기간').fillna(0)
    df4= df3.merge(df4,how='outer', on='기간').fillna(0)
    df5= df4.merge(df5,how='outer', on='기간').fillna(0)
    
    return df5

In [7]:
scale_target = '테이트' #스케일 중심 target
kw_list = [[scale_target]]
app_list = []
target_list = np.delete(train['브랜드'].dropna().unique(),np.where(train['브랜드'].dropna().unique()==scale_target))

for i in range(0,len(target_list),4):
    app_list=[x.replace(',','').strip() for x in target_list[i:i+4]]
    #app_list=[x for x in target_list[i:i+4]]
    if len(app_list)<4:
        while len(app_list) != 4 :
            app_list.append('테스트')
            
    kw_list.append(app_list)

In [8]:
kw_list

[['테이트'],
 ['오모떼', 'CERINI by PAT', '보코', '크로커다일'],
 ['에코라믹', '그렉노먼', '뽕셰프', '멋진밥상'],
 ['비버리힐스폴로클럽', '우리바다', '푸마', 'LG'],
 ['마리노블', '헤스티지', '로베르타 디 까메리노', '레이프릴'],
 ['벨라홈', '종근당건강', 'NNF', '쿠첸'],
 ['린나이', '하늘내린', 'SPC삼립', '바다먹자'],
 ['키친플라워', '피시원', '에펨', '천염담아'],
 ['엘렌실라', '로베르타', '한일', '쿠쿠'],
 ['청정수산', '영산포숙성', '노비타', '제주농장'],
 ['코이모', '소들녘', '스포메틱스', '가이거'],
 ['스칼프솔루션', '아문센', '마담팰리스', 'K-SWISS'],
 ['팽현숙', '마리끌레르', '아리스토우', '스위스밀리터리'],
 ['도네이', '쿠미투니카', '농협', '메디쉴드'],
 ['에버라스트', '더블유베일', '카파', '로이몬스터'],
 ['삼성', '임페리얼', '모즈', '베지밀'],
 ['아이오', '궁중', '세인트스코트', '락앤락'],
 ['도스문도스', '에버홈', '몬테밀라노', '한삼인'],
 ['메시제이', '씨엔지코리아', '전철우', '수협'],
 ['올리고', '컬럼비아', '남영비비안', '헤스떼벨'],
 ['천연담아', '제주바다', '크리스티나앤코', '대우전자'],
 ['마스케라', '해피콜', '개미식품', '크린조이'],
 ['스텐락', '쓰임', '로벤탈', '장보고'],
 ['클레린', '엘르', '세균싹', '보루네오'],
 ['아키', '이롬', '아이넥스', '파시노'],
 ['월드컵', '세렌셉템버', '루이바셋', '달바'],
 ['삼익가구', '까사마루행거', '칼리베이직', '프로피쿡'],
 ['키친아트', '바로바로', '올바로바른자세밴드', '아가타'],
 ['선일금고', '에이치엘사이언스', '한샘', '김정문의섬진강재첩'],
 ['에르나벨

In [23]:
def datalab_api_kw_list(kw_list):
    
    df=datalab_api(kw_list[0]+kw_list[1])
    a, b = df.iloc[:,1].min(), df.iloc[:,1].max()
    print('a=',a,'b=',b)
    i=2
    while i < len(kw_list): 
        try: 
            df1 = datalab_api(kw_list[0]+kw_list[i])
            x,y = df1.iloc[:,1].min(), df1.iloc[:,1].max()
            print('x=',x,'y=',y)
            df1.iloc[:,2] = (df1.iloc[:,2]-x) / (y-x)  * (b-a) + a
            df1.iloc[:,3] = (df1.iloc[:,3]-x) / (y-x)  * (b-a) + a
            df1.iloc[:,4] = (df1.iloc[:,4]-x) / (y-x)  * (b-a) + a
            df1.iloc[:,5] = (df1.iloc[:,5]-x) / (y-x)  * (b-a) + a
            df1 = df1.drop(columns=['기간',scale_target]) # train = 니트. test= 셔츠
            df = pd.concat([df,df1],axis=1)
            i+=1 
            
            df = df[df.columns.drop(list(df.filter(regex='테스트')))]
        except:
            continue

    return df

In [ ]:
df_train = datalab_api_kw_list(kw_list)

a= 14.68812 b= 72.90409
x= 20.14719 y= 100.0
x= 0.96603 y= 4.79488
x= 20.14719 y= 100.0
x= 1.33106 y= 6.60669
x= 4.92688 y= 24.45444


## 2. trend 유형2

#### item_w_mean
- 기간: 2019/05/03~2019/06/01
- 기간 중 아이템의 정규화된 검색량의 가중평균 값(최근일수록 가중치를 높게 부여)

In [3]:
train['날짜'] = train.방송일시.apply(lambda x: x.split(' ')[0])
train['날짜'] = pd.to_datetime(train['날짜'], format='%Y-%m-%d', errors='raise')

test['날짜'] = test.방송일시.apply(lambda x: x.split(' ')[0])
test['날짜'] = pd.to_datetime(test['날짜'], format='%Y-%m-%d', errors='raise')

In [4]:
# start data: -31
# end date: -1
result_train = train.groupby(['날짜','item'])['item'].agg(['count']).reset_index()
result_test = test.groupby(['날짜','item'])['item'].agg(['count']).reset_index()

In [5]:
print('train: ', len(result_train), 'test: ', len(result_test))

train:  7299 test:  582


In [6]:
from datetime import datetime, timedelta

result_train['startDate'] = result_train.날짜.apply(lambda x: x - timedelta(days = 31))
result_train['endDate'] = result_train.날짜.apply(lambda x: x - timedelta(days = 1))

result_test['startDate'] = result_test.날짜.apply(lambda x: x - timedelta(days = 31))
result_test['endDate'] = result_test.날짜.apply(lambda x: x - timedelta(days = 1))

In [7]:
result_train['startDate'] = result_train.startDate.apply(lambda x: str(x).split(' ')[0])
result_train['endDate'] = result_train.endDate.apply(lambda x: str(x).split(' ')[0])

result_test['startDate'] = result_test.startDate.apply(lambda x: str(x).split(' ')[0])
result_test['endDate'] = result_test.endDate.apply(lambda x: str(x).split(' ')[0])

In [33]:
## new 코드
import os
import sys
import urllib.request


def datalab_api(keywords, startDate, endDate):
    client_id = "zkIiC4UfvU1jEWWyp8_2"
    client_secret = "enaM8m_1z7"
    url = "https://openapi.naver.com/v1/datalab/search";
    body = {"startDate":startDate
    ,"endDate": endDate
    ,"timeUnit":"date"
    ,"keywordGroups":[
        {"groupName":str(keywords),"keywords":[str(keywords)]}
    ]
    };
    body = json.dumps(body)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_obj = json.loads(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
        
    df1 = pd.DataFrame(json_obj['results'][0]['data']);
    df1.columns=['기간',str(keywords)]; 
    
    return df1

In [9]:
result_train.head()

,날짜,item,count,startDate,endDate
0,2019-01-01,갈비탕,3,2018-12-01,2018-12-31
1,2019-01-01,꼬막,3,2018-12-01,2018-12-31
2,2019-01-01,남성기초화장품,3,2018-12-01,2018-12-31
3,2019-01-01,냄비,3,2018-12-01,2018-12-31
4,2019-01-01,니트,9,2018-12-01,2018-12-31


In [10]:
result_test.head()

,날짜,item,count,startDate,endDate
0,2020-06-01,TV,9,2020-05-01,2020-05-31
1,2020-06-01,냄비,3,2020-05-01,2020-05-31
2,2020-06-01,돔,3,2020-05-01,2020-05-31
3,2020-06-01,란쥬,3,2020-05-01,2020-05-31
4,2020-06-01,볼륨스타일러,3,2020-05-01,2020-05-31


In [41]:
from tqdm import tqdm
start_number = 6534 #자기가 시작하는 번호
end_number = len(result_train) #끝나는 번호
li = []

for i in tqdm(range(start_number, end_number)):
    try:
        trend = datalab_api(result_train.item[i], result_train.startDate[i], result_train.endDate[i])
        trend.iloc[:,1] = trend.iloc[:,1] / 100

        w_mean = (sum(trend.iloc[0:5,1] * 0.1)+sum(trend.iloc[5:10,1] * 0.2)+sum(trend.iloc[10:15,1] * 0.3)+sum(trend.iloc[15:20,1] * 0.5)+sum(trend.iloc[20:25,1] * 0.7)+sum(trend.iloc[25:31,1] * 0.9))/13.4
        mean_ = trend.iloc[:,1].mean()

        li.append([result_train.날짜[i], result_train.item[i], mean_, w_mean, (w_mean+mean_)/2])
    except:
        continue




  0%|                                                                                          | 0/765 [00:00<?, ?it/s]


  0%|                                                                                  | 1/765 [00:00<01:30,  8.40it/s]


  0%|▏                                                                                 | 2/765 [00:00<01:41,  7.55it/s]


  0%|▎                                                                                 | 3/765 [00:00<01:37,  7.78it/s]


  1%|▍                                                                                 | 4/765 [00:00<01:40,  7.56it/s]


  1%|▌                                                                                 | 5/765 [00:00<01:44,  7.27it/s]


  1%|▋                                                                                 | 6/765 [00:00<01:40,  7.56it/s]


  1%|▊                                                                                 | 7/765 [00:00<01:39,  7.65it/s]


  1%|▊               

 18%|██████████████                                                                  | 134/765 [00:18<01:28,  7.15it/s]


 18%|██████████████                                                                  | 135/765 [00:18<01:27,  7.24it/s]


 18%|██████████████▏                                                                 | 136/765 [00:18<01:28,  7.09it/s]


 18%|██████████████▎                                                                 | 137/765 [00:18<01:29,  7.01it/s]


 18%|██████████████▍                                                                 | 138/765 [00:18<01:27,  7.13it/s]


 18%|██████████████▌                                                                 | 139/765 [00:18<01:28,  7.10it/s]


 18%|██████████████▋                                                                 | 140/765 [00:19<01:24,  7.36it/s]


 18%|██████████████▋                                                                 | 141/765 [00:19<01:24,  7.39it/s]


 19%|██████████████▊    

 35%|████████████████████████████                                                    | 268/765 [00:36<01:06,  7.50it/s]


 35%|████████████████████████████▏                                                   | 269/765 [00:36<01:09,  7.14it/s]


 35%|████████████████████████████▏                                                   | 270/765 [00:37<01:12,  6.79it/s]


 35%|████████████████████████████▎                                                   | 271/765 [00:37<01:12,  6.83it/s]


 36%|████████████████████████████▍                                                   | 272/765 [00:37<01:10,  6.98it/s]


 36%|████████████████████████████▌                                                   | 273/765 [00:37<01:08,  7.19it/s]


 36%|████████████████████████████▋                                                   | 274/765 [00:37<01:06,  7.38it/s]


 36%|████████████████████████████▊                                                   | 275/765 [00:37<01:04,  7.58it/s]


 36%|███████████████████

 53%|██████████████████████████████████████████                                      | 402/765 [00:55<00:58,  6.19it/s]


 53%|██████████████████████████████████████████▏                                     | 403/765 [00:55<00:56,  6.45it/s]


 53%|██████████████████████████████████████████▏                                     | 404/765 [00:55<00:53,  6.81it/s]


 53%|██████████████████████████████████████████▎                                     | 405/765 [00:55<00:51,  6.95it/s]


 53%|██████████████████████████████████████████▍                                     | 406/765 [00:55<00:49,  7.23it/s]


 53%|██████████████████████████████████████████▌                                     | 407/765 [00:56<00:49,  7.27it/s]


 53%|██████████████████████████████████████████▋                                     | 408/765 [00:56<00:48,  7.34it/s]


 53%|██████████████████████████████████████████▊                                     | 409/765 [00:56<00:48,  7.36it/s]


 54%|███████████████████

 70%|████████████████████████████████████████████████████████                        | 536/765 [01:14<00:30,  7.43it/s]


 70%|████████████████████████████████████████████████████████▏                       | 537/765 [01:14<00:30,  7.47it/s]


 70%|████████████████████████████████████████████████████████▎                       | 538/765 [01:14<00:31,  7.22it/s]


 70%|████████████████████████████████████████████████████████▎                       | 539/765 [01:14<00:30,  7.41it/s]


 71%|████████████████████████████████████████████████████████▍                       | 540/765 [01:14<00:30,  7.33it/s]


 71%|████████████████████████████████████████████████████████▌                       | 541/765 [01:14<00:30,  7.43it/s]


 71%|████████████████████████████████████████████████████████▋                       | 542/765 [01:14<00:29,  7.54it/s]


 71%|████████████████████████████████████████████████████████▊                       | 543/765 [01:14<00:28,  7.68it/s]


 71%|███████████████████

 88%|██████████████████████████████████████████████████████████████████████          | 670/765 [01:31<00:12,  7.55it/s]


 88%|██████████████████████████████████████████████████████████████████████▏         | 671/765 [01:31<00:12,  7.58it/s]


 88%|██████████████████████████████████████████████████████████████████████▎         | 672/765 [01:32<00:12,  7.58it/s]


 88%|██████████████████████████████████████████████████████████████████████▍         | 673/765 [01:32<00:12,  7.56it/s]


 88%|██████████████████████████████████████████████████████████████████████▍         | 674/765 [01:32<00:11,  7.74it/s]


 88%|██████████████████████████████████████████████████████████████████████▌         | 675/765 [01:32<00:11,  7.69it/s]


 88%|██████████████████████████████████████████████████████████████████████▋         | 676/765 [01:32<00:11,  7.82it/s]


 88%|██████████████████████████████████████████████████████████████████████▊         | 677/765 [01:32<00:11,  7.67it/s]


 89%|███████████████████

In [40]:
result_train.iloc[6533,]

날짜           2019-11-22 00:00:00
item                         세탁기
count                          3
startDate             2019-10-22
endDate               2019-11-21
Name: 6533, dtype: object

In [42]:
df = pd.DataFrame(li, columns = ['날짜', 'item', 'item_mean', 'item_w_mean', 'item_total_mean'])
df.head()

,날짜,item,item_mean,item_w_mean,item_total_mean
0,2019-11-22,오징어,0.888212,0.946172,0.917192
1,2019-11-22,자켓,0.726972,0.668803,0.697887
2,2019-11-22,청소기,0.799433,0.863131,0.831282
3,2019-11-22,카페트매트,0.718447,0.785152,0.751799
4,2019-11-22,트랙수트,0.527240,0.525926,0.526583


In [43]:
df.tail()

,날짜,item,item_mean,item_w_mean,item_total_mean
753,2019-12-31,침대,0.797164,0.860666,0.828915
754,2019-12-31,코트,0.698028,0.714868,0.706448
755,2019-12-31,트렁크,0.805152,0.885873,0.845512
756,2019-12-31,패딩,0.575793,0.498129,0.536961
757,2020-01-01,압력밥솥,0.832500,0.866014,0.849257


In [44]:
df.to_csv('trend_data_2_train.csv',encoding='cp949', index = False)